In [9]:
import pandas as pd
import json 
from tqdm import tqdm


def to_df(json_file):
    dataset = json.load(open(json_file))
    title, context, question, id, is_impossible, answer = [], [], [], [], [], []
    for topic in dataset['data']:
        for qas in topic['paragraphs']:
            for pair in qas['qas']:
                title.append(topic['title'])
                context.append(qas['context'])
                question.append(pair['question'])
                id.append(pair['id'])
                try:
                    is_impossible.append(pair['is_impossible'])
                except:
                    is_impossible.append(False)
                if not is_impossible[-1]: 
                    answer.append(pair['answers'])
                else: 
                    answer.append([])  
    return pd.DataFrame(zip(title, context, question, id, answer, is_impossible), columns=['title', 'context', 'question', 'id', 'answer', 'is_impossible'])
                

In [10]:
def convert_to_json(df):
    json_file = {}
    json_file['version'] = 'viquad2_training_set'
    json_file['data'] = []
    for id in tqdm(range(len(df))):
        data = df.loc[id]
        question = {}
        question['question'] = data['question']
        if data['is_impossible']:
            question['answers'] = []
        else:
            question['answers'] = data['answer']
        question['id'] = data['id']
        question['is_impossible'] = bool(data['is_impossible'])
        if len(json_file['data']) == 0:
            topic = {}
            topic['title'] = data['title']
            topic['paragraphs'] = [{'qas':[question], 'context':data['context']}]
            json_file['data'].append(topic)
        else:
            if json_file['data'][-1]['title'] == data['title']:
                    if json_file['data'][-1]['paragraphs'][-1]['context'] == data['context']:
                        json_file['data'][-1]['paragraphs'][-1]['qas'].append(question)
                    else:
                        json_file['data'][-1]['paragraphs'].append({'qas':[question], 'context':data['context']})
            else:
                topic = {}
                topic['title'] = data['title']
                topic['paragraphs'] = [{'qas':[question], 'context':data['context']}]
                json_file['data'].append(topic)
    return json_file


In [15]:
train_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD.json')
val_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v1_dev_ViQuAD.json')
test_df = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v1_test_ViQuAD.json')

In [16]:
train_df_new = to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD_new.json')

In [17]:
train_df

,title,context,question,id,answer,is_impossible
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",False
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,"[{'answer_start': 60, 'text': 'Thủ tướng'}]",False
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,"[{'answer_start': 245, 'text': 'Thủ tướng Chín...",False
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng trước k...,uit_000004,[],True
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,[],True
...,...,...,...,...,...,...
28452,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Thời kì Liên hiệp Pháp lãnh đạo kết thúc vào t...,uit_028453,"[{'answer_start': 0, 'text': 'Năm 1958'}]",False
28453,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Các nước nào đã ký kết hiệp ước với Pháp để tu...,uit_028454,"[{'answer_start': 570, 'text': 'Algérie, Tunis...",False
28454,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Quốc gia nào tiếp tục sử dụng biện pháp bạo lự...,uit_028455,"[{'answer_start': 288, 'text': 'Algérie'}]",False
28455,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Điều 68 Hiến pháp Pháp (năm 1958) quy định nhữ...,uit_028456,[],True


In [19]:
train_df['is_impossible'].value_counts()

False    19240
True      9217
Name: is_impossible, dtype: int64

In [20]:
train_df_new['is_impossible'].value_counts()

False    19240
True      3217
Name: is_impossible, dtype: int64

In [21]:
test_df

,title,context,question,id,answer,is_impossible
0,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,uit_01__00061_0_1,"[{'answer_start': 124, 'text': 'Chiến tranh th...",False
1,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Nước Đức hiện nay sự hợp thành của hai nước nà...,uit_01__00061_0_2,"[{'answer_start': 432, 'text': 'Cộng hòa Liên ...",False
2,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,uit_01__00061_0_3,"[{'answer_start': 311, 'text': 'Chiến tranh th...",False
3,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,uit_01__00061_0_4,"[{'answer_start': 480, 'text': 'Năm 1990'}, {'...",False
4,Đức,"Trong thế kỷ XXI, Đức là một đại cường quốc và...","Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...",uit_01__00061_1_1,"[{'answer_start': 223, 'text': 'xuất khẩu và n...",False
...,...,...,...,...,...,...
2205,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,uit_01__00059_13_1,"[{'answer_start': 102, 'text': 'được gửi qua c...",False
2206,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Cần làm gì để tăng tốc độ trả lời cho người ch...,uit_01__00059_13_2,"[{'answer_start': 431, 'text': 'các trang được...",False
2207,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,uit_01__00059_13_3,"[{'answer_start': 251, 'text': 'kết xuất trang...",False
2208,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,Để giải quyết các yêu cầu thì máy chủ web có n...,uit_01__00059_13_4,"[{'answer_start': 304, 'text': 'gửi lại những ...",False


In [22]:
negative_value = train_df[train_df['is_impossible']==True].sample(2000)

In [23]:
negative_value

,title,context,question,id,answer,is_impossible
11956,George Washington,Nhiều nơi và nhiều thực thể đã được đặt tên Wa...,Tiểu bang có đặc điểm gì đặc biệt?,uit_011957,[],True
19807,Đảo Ireland,"Đến cuối thập niên 1960, bất bình của phái dân...",Lục quân Cộng hoà Ireland Lâm thời bán quân sự...,uit_019808,[],True
6577,Giáo hoàng Gioan Phaolô II,Gioan Phaolô II đã biến những chuyến chuyến tô...,Không những làm được nhiều điều tốt mà ông còn...,uit_006578,[],True
22726,Cuba,"Năm 1956 một đảng của những người nổi dậy, gồm...",Mối quan hệ giữa Batista và Thượng nghị sĩ Raf...,uit_022727,[],True
2862,Thổ Nhĩ Kỳ,Thổ Nhĩ Kỳ là một quốc gia thế tục không có qu...,Khi nào thì vị thế của các tôn giáo được phép ...,uit_002863,[],True
...,...,...,...,...,...,...
25332,Trung Quốc,"Tính đến tháng 8 năm 2018, nước Cộng hòa Nhân ...",Cộng hòa Nhân dân Trung Hoa có bao nhiêu thành...,uit_025333,[],True
15256,Bạc Liêu,"Trong giai đoạn 6 tháng đầu năm 2012, Tuy có s...",GDP trong nửa đầu năm 2012 thay đổi bao nhiêu ...,uit_015257,[],True
22932,Cuba,"Cuba là một quần đảo nằm tại Biển Caribbean, v...",Xét về diện tích thì biển Cuba lớn thứ mấy trê...,uit_022933,[],True
4895,Hà Nam (Trung Quốc),Văn hóa Nhị Lý Đầu đã tạo ra bước nhảy từ thời...,Khoảng thời gian tồn tại của liên minh Nhị Lý ...,uit_004896,[],True


In [24]:
train_df.drop(negative_value.index, inplace=True)
train_df = train_df.reset_index(drop=True)

In [25]:
train_df

,title,context,question,id,answer,is_impossible
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",False
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,"[{'answer_start': 60, 'text': 'Thủ tướng'}]",False
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,"[{'answer_start': 245, 'text': 'Thủ tướng Chín...",False
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,[],True
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1987-2000, Phạm Văn Đồng nắm giữ...",uit_000006,[],True
...,...,...,...,...,...,...
26452,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Chính phủ các quốc gia từng là thuộc địa của P...,uit_028452,"[{'answer_start': 467, 'text': 'tiếp tục đàm p...",False
26453,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Thời kì Liên hiệp Pháp lãnh đạo kết thúc vào t...,uit_028453,"[{'answer_start': 0, 'text': 'Năm 1958'}]",False
26454,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Các nước nào đã ký kết hiệp ước với Pháp để tu...,uit_028454,"[{'answer_start': 570, 'text': 'Algérie, Tunis...",False
26455,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Quốc gia nào tiếp tục sử dụng biện pháp bạo lự...,uit_028455,"[{'answer_start': 288, 'text': 'Algérie'}]",False


In [26]:
val_df = pd.concat([negative_value[:1000], val_df]).sort_values('id').reset_index(drop=True)

In [27]:
test_df = pd.concat([negative_value[1000:], test_df]).sort_values('id').reset_index(drop=True)

In [28]:
v2_train_new = convert_to_json(train_df)
with open('./MRC_VLSP/v2_train_ViQuAD_new.json', 'w', encoding='utf8') as f:
    json.dump(v2_train_new, f, ensure_ascii=False, indent=4)

100%|██████████| 26457/26457 [00:03<00:00, 8443.82it/s]


In [29]:
to_df('/home/brownyeyes/WorkDirectory/Multidocs-MRC/MRC_VLSP/v2_train_ViQuAD_new.json')

,title,context,question,id,answer,is_impossible
0,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Tên gọi nào được Phạm Văn Đồng sử dụng khi làm...,uit_000001,"[{'answer_start': 507, 'text': 'Lâm Bá Kiệt'}]",False
1,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Phạm Văn Đồng giữ chức vụ gì trong bộ máy Nhà ...,uit_000002,"[{'answer_start': 60, 'text': 'Thủ tướng'}]",False
2,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ...",uit_000003,"[{'answer_start': 245, 'text': 'Thủ tướng Chín...",False
3,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,Hồ Học Lãm giữ chức vụ gì trong bộ máy Nhà nướ...,uit_000005,[],True
4,Phạm Văn Đồng,Phạm Văn Đồng (1 tháng 3 năm 1906 - 29 tháng 4...,"Giai đoạn năm 1987-2000, Phạm Văn Đồng nắm giữ...",uit_000006,[],True
...,...,...,...,...,...,...
26452,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Chính phủ các quốc gia từng là thuộc địa của P...,uit_028452,"[{'answer_start': 467, 'text': 'tiếp tục đàm p...",False
26453,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Thời kì Liên hiệp Pháp lãnh đạo kết thúc vào t...,uit_028453,"[{'answer_start': 0, 'text': 'Năm 1958'}]",False
26454,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Các nước nào đã ký kết hiệp ước với Pháp để tu...,uit_028454,"[{'answer_start': 570, 'text': 'Algérie, Tunis...",False
26455,Chiến tranh Đông Dương,"Năm 1958, Cộng đồng Pháp khai sinh và Liên hiệ...",Quốc gia nào tiếp tục sử dụng biện pháp bạo lự...,uit_028455,"[{'answer_start': 288, 'text': 'Algérie'}]",False


In [30]:
v2_val_new = convert_to_json(val_df)
with open('./MRC_VLSP/v2_dev_ViQuAD.json', 'w', encoding='utf8') as f:
    json.dump(v2_val_new, f, ensure_ascii=False, indent=4)

100%|██████████| 3285/3285 [00:00<00:00, 7551.38it/s]


In [31]:
v2_test = convert_to_json(test_df)

100%|██████████| 3210/3210 [00:00<00:00, 7635.34it/s]


In [32]:
v2_test['version'] = '2.0'
with open('./MRC_VLSP/v2_test_ViQuAD.json', 'w', encoding='utf8') as f:
    json.dump(v2_test, f, ensure_ascii=False, indent=4)

In [33]:
to_df('./MRC_VLSP/v2_test_ViQuAD.json')

,title,context,question,id,answer,is_impossible
0,Phạm Văn Đồng,"Năm 1954, ông được giao nhiệm vụ Trưởng phái đ...",Hội nghị Đông Dương về Genève có tính chất như...,uit_000011,[],True
1,Phạm Văn Đồng,Theo quan điểm của Trung Quốc (tài liệu Bộ Ngo...,Quan điểm của Trung Quốc là gì đối với thư ký ...,uit_000029,[],True
2,Phạm Văn Đồng,Điều chủ yếu phải nhấn mạnh theo tinh thần trê...,Điều gì có thể đẩy nhân dân vào trong tình thế...,uit_000041,[],True
3,Thực vật có hoa,Sự tiến hóa của giới thực vật đã theo xu hướng...,Nhóm thực vật nào sinh trưởng nhanh nhất?,uit_000058,[],True
4,Thực vật có hoa,Các chứng cứ đầu tiên về sự xuất hiện của thực...,Người ta đã phát hiện được các đặc điểm của th...,uit_000065,[],True
...,...,...,...,...,...,...
3205,Chiến tranh Đông Dương,"Tuy nhiên, viện trợ Mỹ vẫn không thể giúp Pháp...",Cuộc biểu tình của Mỹ với Pháp có thuận lợi kh...,uit_028405,[],True
3206,Chiến tranh Đông Dương,"Ngày 22-7-1954, Chủ tịch Hồ Chí Minh ra lời kê...",Mỹ sẽ tổng tuyển cử thống nhất trong vòng bao ...,uit_028417,[],True
3207,Chiến tranh Đông Dương,"Theo kết quả của hiệp định Geneva, Việt Nam đư...",Sự thắng lợi của lực lượng nào là nguyên nhân ...,uit_028429,[],True
3208,Chiến tranh Đông Dương,Hội nghị tuyên bố chấm dứt chiến tranh ở Lào v...,Lào và Myanmar có gia nhập liên minh quân sự n...,uit_028435,[],True
